In [1]:
import pandas as pd
import math
from statistics import mode 
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import metrics
import math
import graphviz 
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from IPython.display import Image  
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import pydotplus
from sklearn.model_selection import GridSearchCV

In [2]:
df=pd.read_csv("C:\\Users\\Trivikram\\Machine_Learning\\Data\\adult.data",header=None,names=['Age','Workplace','fnlwgt','education','education num','marital-stauts','occupation','relationship','race','sex','capital gain','capital loss','hours per week','native country','Salary'])
df

,Age,Workplace,fnlwgt,education,education num,marital-stauts,occupation,relationship,race,sex,capital gain,capital loss,hours per week,native country,Salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [3]:
def removeQuestionMark(outCol,inpCol,X,y):
    for i in inpCol:
        repl=X[i].value_counts().keys().tolist()[0]
        X[i]=X[i].replace(to_replace=' ?',value=repl)
    rep=y[outCol[0]].value_counts().keys().tolist()[0]
    y[outCol[0]]=y[outCol[0]].replace(to_replace=' ?',value=rep)
    return[X,y]

In [4]:
y=df[['Salary']]
X=df[['Age','fnlwgt','education num','capital gain','capital loss','hours per week','Workplace','education','race','sex','marital-stauts','occupation','relationship','native country']]
inpCol=list(X.columns)
outCol=list(y.columns)
(X,y)=removeQuestionMark(outCol,inpCol,X,y)
categCol=['Workplace','education','race','sex','marital-stauts','occupation','relationship','native country']
contCol=['Age','fnlwgt','education num','capital gain','capital loss','hours per week']

C:\Users\Trivikram\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Trivikram\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
X,y=removeQuestionMark(outCol,inpCol,X,y)
CategData=X.select_dtypes(include='object')
CategCol=CategData.columns
ContData=X.select_dtypes(exclude='object')
contCol=ContData.columns
enc = OneHotEncoder(handle_unknown='ignore')
binaryValues=enc.fit_transform(X[categCol]).toarray()
newCol=list(enc.get_feature_names(categCol))
categData = pd.DataFrame(binaryValues,columns=newCol,index=X.index)
X=categData.merge(df[contCol],left_index=True,right_index=True)
classNames=y['Salary'].value_counts().keys().tolist()

C:\Users\Trivikram\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Trivikram\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
pca = PCA()
dt = DecisionTreeClassifier(criterion='entropy',random_state=42)
pipe = Pipeline(steps=[('pca', pca)])
param_grid = {'pca__n_components': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]}
grid_search = GridSearchCV(estimator = pipe, param_grid = param_grid,cv = 5,verbose=10)
grid_search.fit(X,y)
pipe=grid_search.best_estimator_
print(pipe)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] pca__n_components=0.1 ...........................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............ pca__n_components=0.1, score=-841.870, total=   0.5s
[CV] pca__n_components=0.1 ...........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s


[CV] ............ pca__n_components=0.1, score=-848.251, total=   0.5s
[CV] pca__n_components=0.1 ...........................................


...

[CV] ............ pca__n_components=0.2, score=-850.777, total=   0.4s
[CV] pca__n_components=0.2 ...........................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    3.9s remaining:    0.0s


[CV] ............ pca__n_components=0.2, score=-840.225, total=   0.4s
[CV] pca__n_components=0.3 ...........................................
[CV] ............ pca__n_components=0.3, score=-841.870, total=   0.4s
[CV] pca__n_components=0.3 ...........................................
[CV] ............ pca__n_components=0.3, score=-848.251, total=   0.4s
[CV] pca__n_components=0.3 ...........................................
[CV] ............ pca__n_components=0.3, score=-847.093, total=   0.4s
[CV] pca__n_components=0.3 ...........................................
[CV] ............ pca__n_components=0.3, score=-850.777, total=   0.4s
[CV] pca__n_components=0.3 ...........................................
[CV] ............ pca__n_components=0.3, score=-840.225, total=   0.4s
[CV] pca__n_components=0.4 ...........................................
[CV] ............ pca__n_components=0.4, score=-841.870, total=   0.4s
[CV] pca__n_components=0.4 ...........................................
[CV] .

[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   23.0s finished


Pipeline(memory=None,
         steps=[('pca',
                 PCA(copy=True, iterated_power='auto', n_components=1,
                     random_state=None, svd_solver='auto', tol=0.0,
                     whiten=False))],
         verbose=False)


In [8]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_pca__n_components,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.402423,0.028522,0.047203,0.007167,0.1,{'pca__n_components': 0.1},-841.869812,-848.251351,-847.093126,-850.777218,-840.225284,-845.643242,3.97127,2
1,0.377422,0.012925,0.044403,0.004454,0.2,{'pca__n_components': 0.2},-841.869812,-848.251351,-847.093126,-850.777218,-840.225284,-845.643242,3.97127,2
2,0.365821,0.004167,0.041202,0.003710,0.3,{'pca__n_components': 0.3},-841.869812,-848.251351,-847.093126,-850.777218,-840.225284,-845.643242,3.97127,2
3,0.366621,0.010726,0.037402,0.001855,0.4,{'pca__n_components': 0.4},-841.869812,-848.251351,-847.093126,-850.777218,-840.225284,-845.643242,3.97127,2
4,0.354620,0.008846,0.040602,0.005123,0.5,{'pca__n_components': 0.5},-841.869812,-848.251351,-847.093126,-850.777218,-840.225284,-845.643242,3.97127,2
5,0.343820,0.004445,0.039002,0.004858,0.6,{'pca__n_components': 0.6},-841.869812,-848.251351,-847.093126,-850.777218,-840.225284,-845.643242,3.97127,2
6,0.404023,0.043315,0.053803,0.014730,0.7,{'pca__n_components': 0.7},-841.869812,-848.251351,-847.093126,-850.777218,-840.225284,-845.643242,3.97127,2
7,0.433025,0.192219,0.048603,0.010405,0.8,{'pca__n_components': 0.8},-841.869812,-848.251351,-847.093126,-850.777218,-840.225284,-845.643242,3.97127,2
8,0.521030,0.194949,0.039602,0.008015,0.9,{'pca__n_components': 0.9},-841.869812,-848.251351,-847.093126,-850.777218,-840.225284,-845.643242,3.97127,2
9,0.533631,0.153645,0.053403,0.018063,1,{'pca__n_components': 1},-841.869812,-848.251351,-847.093126,-850.777218,-840.225284,-845.643242,3.97127,1


In [21]:
nComponent=[]
nComponent.append(0)
for i in range(0,10):
    nComponent.append(float(nComponent[i]+0.1))
print(nComponent)

[0, 0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6, 0.7, 0.7999999999999999, 0.8999999999999999, 0.9999999999999999]


In [22]:
for i in nComponent:
    pca = PCA(n_components=i)
    pca.fit(X,y)
    print(pca.explained_variance_)

[]
[1.11407978e+10]
[1.11407978e+10]
[1.11407978e+10]
[1.11407978e+10]
[1.11407978e+10]
[1.11407978e+10]
[1.11407978e+10]
[1.11407978e+10]
[1.11407978e+10]
[1.11407978e+10 5.45426939e+07 1.62198337e+05 1.86106507e+02
 1.48401472e+02 6.44386040e+00 7.31053325e-01 3.12877103e-01
 2.63287445e-01 2.16368498e-01 1.97592239e-01 1.82512673e-01
 1.77667940e-01 1.53971266e-01 1.45573462e-01 1.14914818e-01
 1.11856616e-01 1.06701267e-01 9.46348096e-02 8.98139852e-02
 8.55360734e-02 8.22174448e-02 6.92538587e-02 6.26633289e-02
 5.70347938e-02 5.12816638e-02 4.79610596e-02 4.56475224e-02
 4.39367989e-02 4.26076425e-02 3.70975808e-02 3.66033423e-02
 3.46549659e-02 3.38356860e-02 3.30885118e-02 3.12980681e-02
 3.04578726e-02 2.88075512e-02 2.59840034e-02 2.52621677e-02
 1.97242270e-02 1.85876276e-02 1.73646490e-02 1.53058109e-02
 1.45555809e-02 1.42777666e-02 1.36093860e-02 1.28204827e-02
 9.54661744e-03 8.74666449e-03 6.11815210e-03 5.13576642e-03
 4.79129263e-03 4.06447900e-03 3.58938209e-03 3.402

In [58]:
pca = PCA(n_components=0.6)
pca.fit(X)
max(pca.components_)

array([-1.20604101e-08, -6.56996113e-09,  6.98779215e-10,  1.80422674e-07,
       -4.24468368e-08, -9.92578087e-08, -2.01877984e-08, -5.98637356e-10,
        1.81435096e-08,  1.66808413e-08,  1.11240286e-08,  2.29366307e-08,
        3.91710829e-08, -3.02608640e-09,  1.80050573e-08,  1.07237404e-08,
       -2.98781804e-08, -2.54273407e-08, -3.50614378e-09, -6.93731930e-09,
       -4.71473394e-08,  6.48297042e-09, -6.53320694e-09, -2.08122442e-08,
       -5.91121184e-08, -8.54635371e-08,  3.29282408e-07,  5.48793562e-09,
       -1.90194688e-07, -1.19724593e-07,  1.19724593e-07, -4.86941463e-08,
       -1.02261545e-10, -1.24482284e-07,  3.87571520e-09,  1.65067724e-07,
        4.70926462e-08, -4.27573936e-08,  2.35402674e-08,  6.36336161e-10,
        2.66026126e-08, -6.01286378e-08, -5.36760219e-08,  5.51884091e-08,
        2.90438788e-08, -1.06271802e-08,  4.65352812e-09, -5.68386180e-08,
        2.19376607e-08,  1.11441935e-08,  5.93500058e-09,  2.58857085e-09,
       -9.83256062e-08,  

In [46]:
X.columns

Index(['Workplace_ Federal-gov', 'Workplace_ Local-gov',
       'Workplace_ Never-worked', 'Workplace_ Private',
       'Workplace_ Self-emp-inc', 'Workplace_ Self-emp-not-inc',
       'Workplace_ State-gov', 'Workplace_ Without-pay', 'education_ 10th',
       'education_ 11th',
       ...
       'native country_ Trinadad&Tobago', 'native country_ United-States',
       'native country_ Vietnam', 'native country_ Yugoslavia', 'Age',
       'fnlwgt', 'education num', 'capital gain', 'capital loss',
       'hours per week'],
      dtype='object', length=105)

In [40]:
nComponent=[]
nComponent.append(0.9)
for i in range(0,199):
    nComponent.append(float(nComponent[i]+0.0005))
print(nComponent)

[0.9, 0.9005, 0.9009999999999999, 0.9014999999999999, 0.9019999999999998, 0.9024999999999997, 0.9029999999999997, 0.9034999999999996, 0.9039999999999996, 0.9044999999999995, 0.9049999999999995, 0.9054999999999994, 0.9059999999999994, 0.9064999999999993, 0.9069999999999993, 0.9074999999999992, 0.9079999999999991, 0.9084999999999991, 0.908999999999999, 0.909499999999999, 0.9099999999999989, 0.9104999999999989, 0.9109999999999988, 0.9114999999999988, 0.9119999999999987, 0.9124999999999986, 0.9129999999999986, 0.9134999999999985, 0.9139999999999985, 0.9144999999999984, 0.9149999999999984, 0.9154999999999983, 0.9159999999999983, 0.9164999999999982, 0.9169999999999981, 0.9174999999999981, 0.917999999999998, 0.918499999999998, 0.9189999999999979, 0.9194999999999979, 0.9199999999999978, 0.9204999999999978, 0.9209999999999977, 0.9214999999999977, 0.9219999999999976, 0.9224999999999975, 0.9229999999999975, 0.9234999999999974, 0.9239999999999974, 0.9244999999999973, 0.9249999999999973, 0.92549999

In [41]:
for i in nComponent:
    pca = PCA(n_components=i)
    pca.fit(X)
    print("Min Variation is "+str(i)+" Variation ratio obtained "+str(pca.explained_variance_ratio_))

[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]
[0.99511364]

In [32]:
X

,Workplace_ Federal-gov,Workplace_ Local-gov,Workplace_ Never-worked,Workplace_ Private,Workplace_ Self-emp-inc,Workplace_ Self-emp-not-inc,Workplace_ State-gov,Workplace_ Without-pay,education_ 10th,education_ 11th,...,native country_ Trinadad&Tobago,native country_ United-States,native country_ Vietnam,native country_ Yugoslavia,Age,fnlwgt,education num,capital gain,capital loss,hours per week
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,39,77516,13,2174,0,40
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,50,83311,13,0,0,13
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,38,215646,9,0,0,40
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,53,234721,7,0,0,40
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,28,338409,13,0,0,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,27,257302,12,0,0,38
32557,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,40,154374,9,0,0,40
32558,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,58,151910,9,0,0,40
32559,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,22,201490,9,0,0,20


In [7]:
decreaseList=[]
for i in range(0,10):
    decreaseList.append(round(i*0.1,2))
print(decreaseList)

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
pca = PCA(random_state=42)
dt = DecisionTreeClassifier(random_state=42)
pipe = Pipeline(steps=[('pca',pca),('dt',dt)])
param_grid = {'pca__n_components': [0.015,0.025,0.045,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,0.65,0.7,0.75,0.8,0.85,0.9,0.95]
              ,'dt__min_impurity_decrease':decreaseList}
grid_search = GridSearchCV(estimator = pipe, param_grid = param_grid,cv = 5,verbose=10)
grid_search.fit(X_train,y_train)
pipe=grid_search.best_estimator_
print(pipe)

Fitting 5 folds for each of 220 candidates, totalling 1100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] dt__min_impurity_decrease=0.0, pca__n_components=0.015 ..........
[CV]  dt__min_impurity_decrease=0.0, pca__n_components=0.015, score=0.671, total=   1.0s
[CV] dt__min_impurity_decrease=0.0, pca__n_components=0.015 ..........


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


[CV]  dt__min_impurity_decrease=0.0, pca__n_components=0.015, score=0.669, total=   0.8s
[CV] dt__min_impurity_decrease=0.0, pca__n_components=0.015 ..........


...

[CV]  dt__min_impurity_decrease=0.8, pca__n_components=0.55, score=0.757, total=   0.2s
[CV] dt__min_impurity_decrease=0.8, pca__n_components=0.55 ...........
[CV]  dt__min_impurity_decrease=0.8, pca__n_components=0.55, score=0.757, total=   0.3s
[CV] dt__min_impurity_decrease=0.8, pca__n_components=0.55 ...........
[CV]  dt__min_impurity_decrease=0.8, pca__n_components=0.55, score=0.758, total=   0.3s
[CV] dt__min_impurity_decrease=0.8, pca__n_components=0.55 ...........
[CV]  dt__min_impurity_decrease=0.8, pca__n_components=0.55, score=0.758, total=   0.3s
[CV] dt__min_impurity_decrease=0.8, pca__n_components=0.55 ...........
[CV]  dt__min_impurity_decrease=0.8, pca__n_components=0.55, score=0.758, total=   0.3s
[CV] dt__min_impurity_decrease=0.8, pca__n_components=0.6 ............
[CV]  dt__min_impurity_decrease=0.8, pca__n_components=0.6, score=0.757, total=   0.3s
[CV] dt__min_impurity_decrease=0.8, pca__n_components=0.6 ............
[CV]  dt__min_impurity_decrease=0.8, pca__n_com

[CV]  dt__min_impurity_decrease=0.9, pca__n_components=0.025, score=0.758, total=   0.3s
[CV] dt__min_impurity_decrease=0.9, pca__n_components=0.025 ..........
[CV]  dt__min_impurity_decrease=0.9, pca__n_components=0.025, score=0.758, total=   0.5s
[CV] dt__min_impurity_decrease=0.9, pca__n_components=0.025 ..........
[CV]  dt__min_impurity_decrease=0.9, pca__n_components=0.025, score=0.758, total=   0.3s
[CV] dt__min_impurity_decrease=0.9, pca__n_components=0.045 ..........
[CV]  dt__min_impurity_decrease=0.9, pca__n_components=0.045, score=0.757, total=   0.3s
[CV] dt__min_impurity_decrease=0.9, pca__n_components=0.045 ..........
[CV]  dt__min_impurity_decrease=0.9, pca__n_components=0.045, score=0.757, total=   0.3s
[CV] dt__min_impurity_decrease=0.9, pca__n_components=0.045 ..........
[CV]  dt__min_impurity_decrease=0.9, pca__n_components=0.045, score=0.758, total=   0.3s
[CV] dt__min_impurity_decrease=0.9, pca__n_components=0.045 ..........
[CV]  dt__min_impurity_decrease=0.9, pca

[CV]  dt__min_impurity_decrease=0.9, pca__n_components=0.45, score=0.758, total=   0.3s
[CV] dt__min_impurity_decrease=0.9, pca__n_components=0.5 ............
[CV]  dt__min_impurity_decrease=0.9, pca__n_components=0.5, score=0.757, total=   0.3s
[CV] dt__min_impurity_decrease=0.9, pca__n_components=0.5 ............
[CV]  dt__min_impurity_decrease=0.9, pca__n_components=0.5, score=0.757, total=   0.3s
[CV] dt__min_impurity_decrease=0.9, pca__n_components=0.5 ............
[CV]  dt__min_impurity_decrease=0.9, pca__n_components=0.5, score=0.758, total=   0.3s
[CV] dt__min_impurity_decrease=0.9, pca__n_components=0.5 ............
[CV]  dt__min_impurity_decrease=0.9, pca__n_components=0.5, score=0.758, total=   0.3s
[CV] dt__min_impurity_decrease=0.9, pca__n_components=0.5 ............
[CV]  dt__min_impurity_decrease=0.9, pca__n_components=0.5, score=0.758, total=   0.2s
[CV] dt__min_impurity_decrease=0.9, pca__n_components=0.55 ...........
[CV]  dt__min_impurity_decrease=0.9, pca__n_compone

[Parallel(n_jobs=1)]: Done 1100 out of 1100 | elapsed:  5.3min finished


Pipeline(memory=None,
         steps=[('pca',
                 PCA(copy=True, iterated_power='auto', n_components=0.015,
                     random_state=42, svd_solver='auto', tol=0.0,
                     whiten=False)),
                ('dt',
                 DecisionTreeClassifier(class_weight=None, criterion='gini',
                                        max_depth=None, max_features=None,
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.1,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        presort=False, random_state=42,
                                        splitter='best'))],
         verbose=False)


In [10]:
grid_search.score(X_test,y_test)

0.7631282628723514

In [11]:
dt=DecisionTreeClassifier(class_weight=None, criterion='gini',
                                        max_depth=None, max_features=None,
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.1,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        presort=False, random_state=42,
                                        splitter='best')

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.1, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [12]:
dt.fit(X_train,y_train)
dt.score(X_test,y_test)

0.8134916572832429